In [ ]:
"""
ML Competition: Global Salary Prediction with Cost of Living
V3.1: Optimized ensemble with dynamic weights + advanced techniques
- Scipy optimization for ensemble weights
- Out-of-fold predictions for proper meta-learning
- Additional feature engineering
- Improved regularization
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from pandas.api.types import CategoricalDtype
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

try:
    import lightgbm as lgb
    HAS_LIGHTGBM = True
except:
    HAS_LIGHTGBM = False
    print("Warning: LightGBM not available.")

try:
    import xgboost as xgb
    HAS_XGBOOST = True
except:
    HAS_XGBOOST = False
    print("Warning: XGBoost not available.")

def rmspe(y_true, y_pred):
    """Root Mean Square Percentage Error"""
    y_true = np.maximum(y_true, 1)
    y_pred = np.maximum(y_pred, 1)
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def optimize_weights(predictions, y_true):
    """
    Find optimal weights for ensemble using scipy optimization
    predictions: list of arrays, each with predictions from one model
    y_true: true target values
    """
    n_models = len(predictions)
    
    def objective(weights):
        # Normalize weights
        weights = np.array(weights) / np.sum(weights)
        # Weighted average in log space
        ensemble = np.average(predictions, axis=0, weights=weights)
        # Convert back and calculate error
        ensemble_exp = np.expm1(ensemble)
        ensemble_exp = np.maximum(ensemble_exp, 0)
        y_true_exp = np.expm1(y_true)
        return rmspe(y_true_exp, ensemble_exp)
    
    # Initialize with equal weights
    initial_weights = np.ones(n_models) / n_models
    
    # Constraints: weights must be positive and sum to 1
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1) for _ in range(n_models)]
    
    result = minimize(
        objective,
        initial_weights,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000}
    )
    
    optimal_weights = result.x / np.sum(result.x)
    return optimal_weights, result.fun

# ============================================================================
# 1. DATA LOADING
# ============================================================================
print("="*80)
print("LOADING DATA")
print("="*80)
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
col_data = pd.read_csv('cost_of_living.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Cost of living shape: {col_data.shape}")
print(f"Salary range: ${train['salary_average'].min():.0f} - ${train['salary_average'].max():.0f}")
print(f"Unique cities in train: {train['city'].nunique()}")
print(f"Unique cities in test: {test['city'].nunique()}")

# ============================================================================
# 2. ENHANCED PRE-PROCESSING
# ============================================================================
print("\n" + "="*80)
print("PRE-PROCESSING & MERGING")
print("="*80)

# Smart imputation for cost of living
col_cols = [c for c in col_data.columns if c.startswith('col_')]
print(f"Processing {len(col_cols)} cost-of-living features...")

for col in col_cols:
    col_data[col] = col_data.groupby('country')[col].transform(
        lambda x: x.fillna(x.median()))
    global_median = col_data[col].median()
    col_data[col] = col_data[col].fillna(global_median if not pd.isna(global_median) else 0)

# Merge
train_merged = train.merge(col_data, on=['country', 'state', 'city'], how='left')
test_merged = test.merge(col_data, on=['country', 'state', 'city'], how='left')

print(f"Train after merge: {train_merged.shape}")
print(f"Test after merge: {test_merged.shape}")

# Label encoders
le_dict = {}
for col in ['country', 'state', 'city', 'role']:
    le = LabelEncoder()
    all_vals = pd.concat([
        train_merged[col].astype(str), 
        test_merged[col].astype(str)
    ]).unique()
    le.fit(all_vals)
    le_dict[col] = le

# ============================================================================
# 3. ADVANCED FEATURE ENGINEERING
# ============================================================================
print("\n" + "="*80)
print("FEATURE ENGINEERING")
print("="*80)

def create_target_encodings(train_df):
    """Create target encodings with NO city-level encodings"""
    train_df['log_salary'] = np.log1p(train_df['salary_average'])
    
    # Count features (for confidence weighting)
    country_role_count = train_df.groupby(['country', 'role']).size().to_dict()
    state_role_count = train_df.groupby(['state', 'role']).size().to_dict()
    country_count = train_df.groupby('country').size().to_dict()
    state_count = train_df.groupby('state').size().to_dict()
    role_count = train_df.groupby('role').size().to_dict()
    
    # Country-role (most important for unseen cities)
    country_role_mean = train_df.groupby(['country', 'role'])['log_salary'].mean().to_dict()
    country_role_median = train_df.groupby(['country', 'role'])['log_salary'].median().to_dict()
    country_role_std = train_df.groupby(['country', 'role'])['log_salary'].std().to_dict()
    
    # State-role interactions
    state_role_mean = train_df.groupby(['state', 'role'])['log_salary'].mean().to_dict()
    state_role_median = train_df.groupby(['state', 'role'])['log_salary'].median().to_dict()
    
    # Country level
    country_mean = train_df.groupby('country')['log_salary'].mean().to_dict()
    country_median = train_df.groupby('country')['log_salary'].median().to_dict()
    country_std = train_df.groupby('country')['log_salary'].std().to_dict()
    country_min = train_df.groupby('country')['log_salary'].min().to_dict()
    country_max = train_df.groupby('country')['log_salary'].max().to_dict()
    
    # State level
    state_mean = train_df.groupby('state')['log_salary'].mean().to_dict()
    state_median = train_df.groupby('state')['log_salary'].median().to_dict()
    state_std = train_df.groupby('state')['log_salary'].std().to_dict()
    
    # Role level
    role_mean = train_df.groupby('role')['log_salary'].mean().to_dict()
    role_median = train_df.groupby('role')['log_salary'].median().to_dict()
    role_std = train_df.groupby('role')['log_salary'].std().to_dict()
    role_min = train_df.groupby('role')['log_salary'].min().to_dict()
    role_max = train_df.groupby('role')['log_salary'].max().to_dict()
    
    # Global stats
    global_mean = train_df['log_salary'].mean()
    global_median = train_df['log_salary'].median()
    global_std = train_df['log_salary'].std()
    
    train_df.drop('log_salary', axis=1, inplace=True)
    
    return {
        'country_role_mean': country_role_mean,
        'country_role_median': country_role_median,
        'country_role_std': country_role_std,
        'country_role_count': country_role_count,
        'state_role_mean': state_role_mean,
        'state_role_median': state_role_median,
        'state_role_count': state_role_count,
        'country_mean': country_mean,
        'country_median': country_median,
        'country_std': country_std,
        'country_min': country_min,
        'country_max': country_max,
        'country_count': country_count,
        'state_mean': state_mean,
        'state_median': state_median,
        'state_std': state_std,
        'state_count': state_count,
        'role_mean': role_mean,
        'role_median': role_median,
        'role_std': role_std,
        'role_min': role_min,
        'role_max': role_max,
        'role_count': role_count,
        'global_mean': global_mean,
        'global_median': global_median,
        'global_std': global_std
    }

def engineer_features(df, encodings, le_dict):
    """Enhanced feature engineering focused on generalization"""
    df = df.copy()
    e = encodings
    col_cols = [c for c in df.columns if c.startswith('col_')]
    
    # Label encodings
    df['country_encoded'] = le_dict['country'].transform(df['country'].astype(str))
    df['state_encoded'] = le_dict['state'].transform(df['state'].astype(str))
    df['role_encoded'] = le_dict['role'].transform(df['role'].astype(str))
    
    # === TARGET ENCODINGS (hierarchical fallback) ===
    df['country_role_mean'] = df.apply(
        lambda x: e['country_role_mean'].get(
            (x['country'], x['role']),
            e['country_mean'].get(x['country'], e['global_mean'])
        ), axis=1)
    df['country_role_median'] = df.apply(
        lambda x: e['country_role_median'].get(
            (x['country'], x['role']),
            e['country_median'].get(x['country'], e['global_median'])
        ), axis=1)
    df['country_role_std'] = df.apply(
        lambda x: e['country_role_std'].get(
            (x['country'], x['role']),
            e['country_std'].get(x['country'], e['global_std'])
        ), axis=1)
    
    # Confidence features (sample size)
    df['country_role_count'] = df.apply(
        lambda x: e['country_role_count'].get((x['country'], x['role']), 0), axis=1)
    df['country_role_confidence'] = np.log1p(df['country_role_count'])
    
    # State-Role
    df['state_role_mean'] = df.apply(
        lambda x: e['state_role_mean'].get(
            (x['state'], x['role']),
            e['country_role_mean'].get((x['country'], x['role']), e['global_mean'])
        ), axis=1)
    df['state_role_median'] = df.apply(
        lambda x: e['state_role_median'].get(
            (x['state'], x['role']),
            e['country_role_median'].get((x['country'], x['role']), e['global_median'])
        ), axis=1)
    df['state_role_count'] = df.apply(
        lambda x: e['state_role_count'].get((x['state'], x['role']), 0), axis=1)
    
    # Country stats
    df['country_mean'] = df['country'].map(e['country_mean']).fillna(e['global_mean'])
    df['country_median'] = df['country'].map(e['country_median']).fillna(e['global_median'])
    df['country_std'] = df['country'].map(e['country_std']).fillna(e['global_std'])
    df['country_range'] = df['country'].map(e['country_max']).fillna(e['global_mean']) - \
                          df['country'].map(e['country_min']).fillna(e['global_mean'])
    df['country_count'] = df['country'].map(e['country_count']).fillna(0)
    
    # State stats
    df['state_mean'] = df['state'].map(e['state_mean']).fillna(df['country_mean'])
    df['state_median'] = df['state'].map(e['state_median']).fillna(df['country_median'])
    df['state_std'] = df['state'].map(e['state_std']).fillna(df['country_std'])
    df['state_count'] = df['state'].map(e['state_count']).fillna(0)
    
    # Role stats
    df['role_mean'] = df['role'].map(e['role_mean']).fillna(e['global_mean'])
    df['role_median'] = df['role'].map(e['role_median']).fillna(e['global_median'])
    df['role_std'] = df['role'].map(e['role_std']).fillna(e['global_std'])
    df['role_range'] = df['role'].map(e['role_max']).fillna(e['global_mean']) - \
                       df['role'].map(e['role_min']).fillna(e['global_mean'])
    df['role_count'] = df['role'].map(e['role_count']).fillna(0)
    
    # === COST OF LIVING FEATURES ===
    for col in col_cols:
        df[col] = df[col].fillna(df.groupby('country')[col].transform('median'))
        df[col] = df[col].fillna(df[col].median())
    
    # Basic aggregations
    df['col_mean'] = df[col_cols].mean(axis=1)
    df['col_median'] = df[col_cols].median(axis=1)
    df['col_std'] = df[col_cols].std(axis=1).fillna(0)
    df['col_max'] = df[col_cols].max(axis=1)
    df['col_min'] = df[col_cols].min(axis=1)
    df['col_range'] = df['col_max'] - df['col_min']
    df['col_q75'] = df[col_cols].quantile(0.75, axis=1)
    df['col_q25'] = df[col_cols].quantile(0.25, axis=1)
    df['col_iqr'] = df['col_q75'] - df['col_q25']
    df['col_cv'] = df['col_std'] / (df['col_mean'] + 1e-6)
    df['col_skew'] = df[col_cols].skew(axis=1)
    
    # Specific COL categories
    df['col_housing_proxy'] = df[[c for c in col_cols[:15]]].mean(axis=1)
    df['col_services_proxy'] = df[[c for c in col_cols[15:30]]].mean(axis=1)
    df['col_goods_proxy'] = df[[c for c in col_cols[30:]]].mean(axis=1)
    
    # === KEY INTERACTION FEATURES ===
    # Salary per COL unit
    df['role_mean_per_col'] = df['role_mean'] / (df['col_mean'] + 1e-6)
    df['country_mean_per_col'] = df['country_mean'] / (df['col_mean'] + 1e-6)
    df['country_role_mean_per_col'] = df['country_role_mean'] / (df['col_mean'] + 1e-6)
    df['state_mean_per_col'] = df['state_mean'] / (df['col_mean'] + 1e-6)
    df['state_role_mean_per_col'] = df['state_role_mean'] / (df['col_mean'] + 1e-6)
    
    # COL-adjusted features
    df['col_adjusted_country_role'] = df['country_role_mean'] * np.log1p(df['col_mean'])
    df['col_adjusted_state_role'] = df['state_role_mean'] * np.log1p(df['col_mean'])
    df['col_adjusted_role'] = df['role_mean'] * np.log1p(df['col_mean'])
    
    # Ratios
    df['country_vs_role_ratio'] = df['country_mean'] / (df['role_mean'] + 1e-6)
    df['state_vs_country_ratio'] = df['state_mean'] / (df['country_mean'] + 1e-6)
    df['country_role_vs_role_ratio'] = df['country_role_mean'] / (df['role_mean'] + 1e-6)
    df['state_role_vs_country_role'] = df['state_role_mean'] / (df['country_role_mean'] + 1e-6)
    
    # Deviation features
    df['country_role_vs_country_dev'] = df['country_role_mean'] - df['country_mean']
    df['state_vs_country_dev'] = df['state_mean'] - df['country_mean']
    df['state_role_vs_country_role_dev'] = df['state_role_mean'] - df['country_role_mean']
    
    # Purchasing power proxies
    df['purchasing_power_role'] = df['role_mean'] - np.log1p(df['col_mean'])
    df['purchasing_power_country'] = df['country_mean'] - np.log1p(df['col_mean'])
    df['purchasing_power_country_role'] = df['country_role_mean'] - np.log1p(df['col_mean'])
    
    # Weighted averages (confidence-weighted)
    df['weighted_country_role'] = df['country_role_mean'] * df['country_role_confidence']
    df['weighted_state_role'] = df['state_role_mean'] * np.log1p(df['state_role_count'])
    
    # Variance features
    df['salary_variance_country'] = df['country_std'] ** 2
    df['salary_variance_role'] = df['role_std'] ** 2
    
    # Clean up
    # <<< THIS IS THE FIX >>>
    df = df.replace([np.inf, -np.inf], 0).fillna(0)
    
    return df

# ============================================================================
# 4. PREPARE DATA
# ============================================================================
print("\n" + "="*80)
print("PREPARING DATA")
print("="*80)

valid_idx = train_merged['salary_average'].notna()
train_clean = train_merged[valid_idx].copy()
print(f"Samples after removing NaN targets: {len(train_clean)}")

y_log = np.log1p(train_clean['salary_average'])

# Get feature columns
temp_encodings = create_target_encodings(train_clean.copy())
temp_features = engineer_features(train_clean.head(), temp_encodings, le_dict)
exclude_cols = ['ID', 'country', 'state', 'city', 'role', 'salary_average', 'city_id']
feature_cols = [c for c in temp_features.columns if c not in exclude_cols]
print(f"Total features: {len(feature_cols)}")

# ============================================================================
# 5. CITY-AWARE CROSS-VALIDATION
# ============================================================================
print("\n" + "="*80)
print("TRAINING WITH CITY-AWARE CV + WEIGHT OPTIMIZATION")
print("="*80)

n_folds = 5
gkf = GroupKFold(n_splits=n_folds)
groups = train_clean['city']

# Model configurations
models_config = []

if HAS_LIGHTGBM:
    models_config.append({
        'name': 'LightGBM_v1',
        'model': lgb.LGBMRegressor(
            n_estimators=3000,
            learning_rate=0.008,
            max_depth=8,
            num_leaves=31,
            subsample=0.7,
            colsample_bytree=0.7,
            reg_alpha=0.1,
            reg_lambda=0.1,
            random_state=42,
            n_jobs=-1,
            verbose=-1
        ),
        'cats': ['role_encoded', 'country_encoded', 'state_encoded']
    })
    
    models_config.append({
        'name': 'LightGBM_v2',
        'model': lgb.LGBMRegressor(
            n_estimators=3000,
            learning_rate=0.01,
            max_depth=6,
            num_leaves=15,
            subsample=0.8,
            colsample_bytree=0.6,
            reg_alpha=0.5,
            reg_lambda=0.5,
            random_state=123,
            n_jobs=-1,
            verbose=-1
        ),
        'cats': ['role_encoded', 'country_encoded', 'state_encoded']
    })

if HAS_XGBOOST:
    models_config.append({
        'name': 'XGBoost',
        'model': xgb.XGBRegressor(
            n_estimators=3000,
            learning_rate=0.008,
            max_depth=7,
            subsample=0.7,
            colsample_bytree=0.7,
            reg_alpha=0.1,
            reg_lambda=0.1,
            gamma=0.1,
            random_state=42,
            n_jobs=-1,
            tree_method='hist',
            enable_categorical=True,
            early_stopping_rounds=100
        ),
        'cats': True
    })

models_config.append({
    'name': 'GradientBoosting',
    'model': GradientBoostingRegressor(
        n_estimators=600,
        learning_rate=0.01,
        max_depth=6,
        subsample=0.7,
        max_features='sqrt',
        random_state=42
    ),
    'cats': False
})

# Create categorical dtypes for XGBoost
cat_dtypes = {}
for col_name in ['role', 'country', 'state']:
    encoded_col = f"{col_name}_encoded"
    le = le_dict[col_name]
    all_cats = le.transform(le.classes_)
    cat_dtypes[encoded_col] = CategoricalDtype(
        categories=sorted(np.unique(all_cats)), ordered=False)

# Store OOF predictions for weight optimization
oof_preds_log = [np.zeros(len(train_clean)) for _ in models_config]
all_test_preds_log = [[] for _ in models_config]
model_scores = []

for model_idx, model_config in enumerate(models_config):
    model_name = model_config['name']
    print(f"\n{'-'*80}")
    print(f"Training {model_name}...")
    print(f"{'-'*80}")
    
    fold_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(gkf.split(train_clean, y_log, groups)):
        train_fold = train_clean.iloc[train_idx]
        val_fold = train_clean.iloc[val_idx]
        y_train_log = y_log.iloc[train_idx]
        y_val_log = y_log.iloc[val_idx]
        y_val = np.expm1(y_val_log)
        
        # Create encodings
        encodings = create_target_encodings(train_fold.copy())
        
        # Engineer features
        X_train = engineer_features(train_fold, encodings, le_dict)[feature_cols]
        X_val = engineer_features(val_fold, encodings, le_dict)[feature_cols]
        X_test = engineer_features(test_merged, encodings, le_dict)[feature_cols]
        
        # Train model
        model = clone(model_config['model'])
        
        if model_name.startswith('LightGBM'):
            model.fit(X_train, y_train_log,
                     eval_set=[(X_val, y_val_log)],
                     categorical_feature=model_config['cats'],
                     callbacks=[lgb.early_stopping(150, verbose=False)])
        elif model_name == 'XGBoost':
            X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
            X_val.columns = X_val.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
            X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
            
            cat_features = [c for c in X_train.columns if c.endswith('_encoded')]
            for col in cat_features:
                if col in cat_dtypes:
                    X_train[col] = X_train[col].astype(cat_dtypes[col])
                    X_val[col] = X_val[col].astype(cat_dtypes[col])
                    X_test[col] = X_test[col].astype(cat_dtypes[col])
            
            model.fit(X_train, y_train_log,
                     eval_set=[(X_val, y_val_log)],
                     verbose=False)
        else:
            model.fit(X_train, y_train_log)
        
        # OOF predictions (in log space)
        val_preds_log = model.predict(X_val)
        oof_preds_log[model_idx][val_idx] = val_preds_log
        
        # Validate
        val_preds = np.expm1(val_preds_log)
        val_preds = np.maximum(val_preds, 0)
        
        score = rmspe(y_val, val_preds)
        fold_scores.append(score)
        print(f"Fold {fold+1}: RMSPE = {score:.6f} "
              f"(held out {len(np.unique(val_fold['city']))} cities)")
        
        # Test predictions
        test_preds_log = model.predict(X_test)
        all_test_preds_log[model_idx].append(test_preds_log)
    
    avg_score = np.mean(fold_scores)
    std_score = np.std(fold_scores)
    print(f"\n{model_name} CV: {avg_score:.6f} (+/- {std_score:.6f})")
    model_scores.append((model_name, avg_score))

# Average test predictions across folds for each model
test_preds_avg = [np.mean(preds, axis=0) for preds in all_test_preds_log]

# ============================================================================
# 6. OPTIMIZE ENSEMBLE WEIGHTS
# ============================================================================
print("\n" + "="*80)
print("OPTIMIZING ENSEMBLE WEIGHTS")
print("="*80)

print("Finding optimal weights using OOF predictions...")
optimal_weights, optimal_score = optimize_weights(oof_preds_log, y_log)

print("\nModel Performance Summary:")
for name, score in sorted(model_scores, key=lambda x: x[1]):
    print(f"  {name}: {score:.6f}")

print(f"\nOptimized ensemble weights:")
for i, (model_config, weight) in enumerate(zip(models_config, optimal_weights)):
    print(f"  {model_config['name']}: {weight:.4f}")

print(f"\nOptimized ensemble OOF score: {optimal_score:.6f}")

# ============================================================================
# 7. CREATE FINAL PREDICTIONS
# ============================================================================
print("\n" + "="*80)
print("CREATING SUBMISSION")
print("="*80)

# Use optimized weights
final_preds_log = np.average(test_preds_avg, axis=0, weights=optimal_weights)
final_preds = np.expm1(final_preds_log)
final_preds = np.maximum(final_preds, 0)

print(f"Prediction range: ${final_preds.min():.2f} - ${final_preds.max():.2f}")

# Create submission
submission = pd.DataFrame({
    'ID': test['ID'],
    'salary_average': final_preds
})
submission.to_csv('submission_v3_1.csv', index=False)
print("\n✓ Submission saved to 'submission_v3_1.csv'")

print("\nSample predictions:")
print(submission.head(20))
print("\n" + "="*80)
print("COMPLETE!")
print("="*80)